In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import hockey_rink
from sklearn.metrics import roc_auc_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import requests
import json


In [11]:
standings_url = "https://api-web.nhle.com/v1/standings/now"

# # Send a GET request to the NHL API
teams_response = requests.get(standings_url)
# Parse the response as a dictionary
teams_response_dict = teams_response.json()
print(teams_response_dict)

{'wildCardIndicator': False, 'standings': [{'clinchIndicator': 'p', 'conferenceAbbrev': 'E', 'conferenceHomeSequence': 1, 'conferenceL10Sequence': 4, 'conferenceName': 'Eastern', 'conferenceRoadSequence': 4, 'conferenceSequence': 1, 'date': '2024-04-18', 'divisionAbbrev': 'M', 'divisionHomeSequence': 1, 'divisionL10Sequence': 4, 'divisionName': 'Metropolitan', 'divisionRoadSequence': 1, 'divisionSequence': 1, 'gameTypeId': 2, 'gamesPlayed': 82, 'goalDifferential': 53, 'goalDifferentialPctg': 0.646341, 'goalAgainst': 229, 'goalFor': 282, 'goalsForPctg': 3.439024, 'homeGamesPlayed': 41, 'homeGoalDifferential': 32, 'homeGoalsAgainst': 110, 'homeGoalsFor': 142, 'homeLosses': 11, 'homeOtLosses': 0, 'homePoints': 60, 'homeRegulationPlusOtWins': 27, 'homeRegulationWins': 24, 'homeTies': 0, 'homeWins': 30, 'l10GamesPlayed': 10, 'l10GoalDifferential': 6, 'l10GoalsAgainst': 30, 'l10GoalsFor': 36, 'l10Losses': 3, 'l10OtLosses': 0, 'l10Points': 14, 'l10RegulationPlusOtWins': 5, 'l10RegulationWins'

In [44]:
# Create an empty list to store the team names
team_name_list = []
# Create an empty list to store the team conference
team_conf_list = []
# Create an empty list to store the team division
team_div_list = []
# Create an empty list to store the team goals for
team_GF_list = []
# Create an empty list to store the team goals Against
team_GA_list = []
# Create an empty list to store the team goals Against
team_GDiff_list = []
# Create an empty list to store the team wins
team_wins_list = []
# Create an empty list to store the team losses
team_losses_list = []
# Create an empty list to store the team losses
team_points_list = []
# Create an empty list to store the team OT losses
team_OTL_list = []


# Create an empty list to store the team division
team_division_list = [] 
#Iterate over the teams in the response to get team_ids
for team in teams_response_dict['standings']:
    # Extract the team name
    team_name=team["teamName"]["default"]
    team_name_list.append(team_name)
    team_conf=team["conferenceName"]
    team_conf_list.append(team_conf)
    team_div=team["divisionName"]
    team_div_list.append(team_div)
    team_wins=team['wins']
    team_wins_list.append(team_wins)
    team_losses=team["losses"]
    team_losses_list.append(team_losses)
    team_GF=team["goalFor"]
    team_GF_list.append(team_GF)
    team_GA=team["goalAgainst"]
    team_GA_list.append(team_GA)
    team_GDiff=team["goalDifferential"]
    team_GDiff_list.append(team_GDiff)
    team_points=team['points']
    team_points_list.append('points')
    team_OTL=team["otLosses"]
    team_OTL_list.append(team_OTL)

    

In [53]:
team_info_dict = {
    'team': team_name_list,
    'conference': team_conf_list,
    'division': team_div_list,
    'wins': team_wins_list,
    'losses': team_losses_list,
    'GF': team_GF_list,
    'GA': team_GA_list,
    'goal_diff': team_GDiff_list,
    'OTL': team_OTL_list
}

In [54]:
team_df= pd.DataFrame(team_info_dict)

In [55]:
team_df

,team,conference,division,wins,losses,GF,GA,goal_diff,OTL
0,New York Rangers,Eastern,Metropolitan,55,23,282,229,53,4
1,Dallas Stars,Western,Central,52,21,298,234,64,9
2,Carolina Hurricanes,Eastern,Metropolitan,52,23,279,216,63,7
3,Winnipeg Jets,Western,Central,52,24,259,199,60,6
4,Florida Panthers,Eastern,Atlantic,52,24,268,200,68,6
5,Vancouver Canucks,Western,Pacific,50,23,279,223,56,9
6,Boston Bruins,Eastern,Atlantic,47,20,267,224,43,15
7,Colorado Avalanche,Western,Central,50,25,304,254,50,7
8,Edmonton Oilers,Western,Pacific,49,27,294,237,57,6
9,Toronto Maple Leafs,Eastern,Atlantic,46,26,303,263,40,10
